In [2]:
pip install pandas==1.3.4

     |████████████████████████████████| 11.5 MB 3.7 MB/s eta 0:00:01    |▊                               | 245 kB 3.7 MB/s eta 0:00:04
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.


In [52]:
%load_ext autoreload
%autoreload 2
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = 'cpu'


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [71]:
# !pip install fcsparser
# !pip install pandas
# !pip install scanpy
# !pip install torch

In [ ]:
# !export TF_BINARY_URL=https://files.pythonhosted.org/packages/dc/65/a94519cd8b4fd61a7b002cb752bfc0c0e5faa25d1f43ec4f0a4705020126/tensorflow-1.15.0-cp37-cp37m-macosx_10_11_x86_64.whl


In [6]:
import pandas as pd
import scanpy as sc
import os
from sklearn.model_selection import train_test_split
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
from torch.autograd import Variable

In [87]:
# IF this doesn't work, try reading fcs natively using python
data_dir = f'/home/razgaon/ondemand/data/sys/myjobs/projects/default/3/'
NSCLC_pickle = f'{data_dir}NSCLC_prepared_data_test.pickle'
PBMC_pickle = f'{data_dir}PBMC_prepared_data_test.pickle'
NSCLC = pd.read_pickle(NSCLC_pickle)
PBMC = pd.read_pickle(PBMC_pickle)


# Try to add an id that maps to the fcs file name


In [88]:
PBMC_df = PBMC.sample(100000)
PBMC_df = NSCLC.sample(100000)

In [89]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
excluded_markers = ['id', 'Time', 'Eventlength', 'sample', 'batch']
NSCLC_markers = [col for col in NSCLC_df.columns if col not in excluded_markers]
NSCLC_df[NSCLC_markers] = scaler.fit_transform(NSCLC_df[NSCLC_markers])

PBMC_markers = [col for col in PBMC_df.columns if col not in excluded_markers]
PBMC_df[PBMC_markers] = scaler.fit_transform(PBMC_df[PBMC_markers])


In [90]:
df = pd.concat([PBMC_df, NSCLC_df])

In [91]:

class MyDataset(Dataset):
 
    def __init__(self,df):
        x=df[markers].values
        self.x_train=torch.tensor(x,dtype=torch.float32)

    def __len__(self):
        return len(self.x_train)
   
    def __getitem__(self,idx):
        return self.x_train[idx]


In [92]:
# Convert metals to their marker names to ensure consistency when merging dataframes


In [93]:
markers = NSCLC_markers + PBMC_markers
dataset = MyDataset(df[markers])
# # Data loader
train_loader = DataLoader(dataset, batch_size=10000, shuffle=True)
test_loader = DataLoader(dataset, batch_size=10000, shuffle=True)



In [94]:
# Demonstration of merging dataframes using concatenation
df1 = pd.DataFrame([[1,2,3,4,5],[6,7,8,9,10]], columns = ["A", "B", "C", "D", "E"])
df2 = pd.DataFrame([[1,2,3,4,5],[6,7,8,9,10]], columns = ["F", "G", "H", "D", "E"])
x = pd.concat([df1, df2])
x



,A,B,C,D,E,F,G,H
0,1.0,2.0,3.0,4,5,NaN,NaN,NaN
1,6.0,7.0,8.0,9,10,NaN,NaN,NaN
0,NaN,NaN,NaN,4,5,1.0,2.0,3.0
1,NaN,NaN,NaN,9,10,6.0,7.0,8.0


In [95]:
class VAE(nn.Module):
    def __init__(self, x_dim, latent_dim=5, hdim=50):
        super(VAE, self).__init__()
        self.x_dim = x_dim
        self.latent_dim = latent_dim
        self.fc1 = nn.Linear(x_dim , hdim)
        self.fc21 = nn.Linear(hdim, latent_dim)
        self.fc22 = nn.Linear(hdim, latent_dim)
        self.fc3 = nn.Linear(latent_dim, hdim)
        self.fc4 = nn.Linear(hdim, x_dim)
        
    def encode(self, x):
        x_ = torch.cat([x], dim=-1)
        h1 = F.relu(self.fc1(x_))
        return self.fc21(h1), self.fc22(h1)
    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return self.fc4(h3)
    def reparameterize(self, mu, var):
#         if self.distance == 'standard':
        if self.training:
            std = var.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            result = eps.mul(std).add_(mu)
            return result
        else:
            return mu
    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, self.x_dim))
        latent = self.reparameterize(mu, logvar)
        reconstructed = self.decode(latent)
        return reconstructed, latent

In [96]:
def compute_kernel(x, y, device):
    x_size = x.to(device).size(0)
    y_size = y.to(device).size(0)
    dim = x.size(1)
    x = x.unsqueeze(1)
    y = y.unsqueeze(0)
    tiled_x = x.expand(x_size, y_size, dim).to(device)
    tiled_y = y.expand(x_size, y_size, dim).to(device)
    kernel_input = (tiled_x - tiled_y).pow(2).mean(2)/float(dim)
    return torch.exp(-kernel_input)

def compute_mmd(x, y, device):
    x_kernel = compute_kernel(x, x, device)
    y_kernel = compute_kernel(y, y, device)
    xy_kernel = compute_kernel(x, y, device)
    mmd = x_kernel.mean() + y_kernel.mean() - 2*xy_kernel.mean()
    return mmd

def vae_loss_function(inputs, encoded, decoded, true_samples, device):
    #reconstruction loss - decoding
    nll = (decoded - inputs).pow(2).mean()
    #distance + reconstruction loss
    mmd = compute_mmd(true_samples, encoded.to(device), device)
    return(nll, mmd)

In [101]:
def train(net, learning_rate, epochs, train_loader, test_loader, optimizer = 'Adam'):
    if optimizer == 'Adam':
        optimizer = optim.Adam(net.parameters(), lr = learning_rate)
    elif optimizer == 'SGD':
        optimizer = optim.SGD(net.parameters(), lr = learning_rate, 
                              momentum = .95, nesterov = True)
    
    for epoch in range(epochs):
        training_loss = 0
        training_reconstruction_error = 0
        training_mmd = 0

        net.train()
        for batchnum, X in enumerate(train_loader):
            optimizer.zero_grad()
            reconstruction, mu = net(X)
            true_samples = torch.randn(mu.shape)
            reconstruction_error, mmd = vae_loss_function(inputs = X, encoded=mu, decoded=reconstruction, true_samples=true_samples , device=device)
            loss = reconstruction_error + mmd
            loss.backward()
            
            optimizer.step()

            training_reconstruction_error += reconstruction_error
            training_mmd  += mmd
            training_loss += loss

        training_reconstruction_error /= (batchnum+1)
        training_mmd  /= (batchnum+1)
        training_loss /= (batchnum+1)
        print('Training loss for epoch %i is %.8f, Reconstruction is %.8f, mmd is %.8f'%(epoch, training_loss, training_reconstruction_error, training_mmd) )
        
        #Testing loop

        testing_reconstruction_error = 0
        testing_mmd = 0

        with torch.no_grad():
            for batchnum, X in enumerate(test_loader):
                reconstruction, mu = net(X)
                true_samples = torch.randn(mu.shape)
                reconstruction_error, mmd = vae_loss_function(inputs = X, encoded=mu, decoded=reconstruction, true_samples=true_samples , device=device)

                testing_reconstruction_error += reconstruction_error
                testing_mmd += mmd
            
            testing_reconstruction_error /= (batchnum+1)
            testing_mmd /= (batchnum+1)
            print('Testing loss for epoch %i is %.8f, Reconstruction is %.8f, mmd is %.8f'%(epoch, testing_reconstruction_error + testing_mmd, testing_reconstruction_error, testing_mmd) )
        

In [102]:
net = VAE(x_dim=dataset.x_train.shape[1])

In [ ]:
train(net, learning_rate = .0001, epochs = 30,
      train_loader = train_loader, test_loader = test_loader)

Training loss for epoch 0 is 0.11626394, Reconstruction is 0.11567972, mmd is 0.00058423
Testing loss for epoch 0 is 0.10477105, Reconstruction is 0.10426947, mmd is 0.00050157
Training loss for epoch 1 is 0.09631941, Reconstruction is 0.09555338, mmd is 0.00076602
Testing loss for epoch 1 is 0.08838500, Reconstruction is 0.08712639, mmd is 0.00125861
Training loss for epoch 2 is 0.08261578, Reconstruction is 0.08079699, mmd is 0.00181878
Testing loss for epoch 2 is 0.07732584, Reconstruction is 0.07493477, mmd is 0.00239107
Training loss for epoch 3 is 0.07329013, Reconstruction is 0.07063015, mmd is 0.00265998
Testing loss for epoch 3 is 0.06939243, Reconstruction is 0.06649832, mmd is 0.00289411
Training loss for epoch 4 is 0.06618740, Reconstruction is 0.06331658, mmd is 0.00287082
Testing loss for epoch 4 is 0.06291854, Reconstruction is 0.06016003, mmd is 0.00275852
Training loss for epoch 5 is 0.06009661, Reconstruction is 0.05760598, mmd is 0.00249063
Testing loss for epoch 5 i